In [1]:
import importlib
import pandas as pd
from tqdm import tqdm
from unidecode import unidecode

In [2]:
import utils.db as db
import utils.clean_data as c

In [67]:
importlib.reload(c)

<module 'utils.clean_data' from '/Users/andreamarin/Documents/personal_projects/keyword-classifier/data_analysis/utils/clean_data.py'>

In [3]:
conn = db.connect_mongo_db("senate-publication")

In [6]:
conn.list_collection_names()

['initiatives', 'publication', 'propositions', 'unique_publications']

In [7]:
collection = conn.get_collection("unique_publications")

In [8]:
collection_columns = collection.find_one().keys()
collection_columns

dict_keys(['_id', 'type', 'original_summary', 'related_publications', 'involved_authors', 'involved_parties', 'related_dates', 'processed_text', 'topics_distribution'])

# Limpiar datos

## Funciones Generales

In [9]:
select_columns = [
    "_id",
    "original_summary",
    "topics_distribution"
]

In [10]:
def clean_datos_inegi(datos_inegi):
    datos_inegi["estado"] = datos_inegi.estado.apply(str.lower)
    datos_inegi["estado"] = datos_inegi.estado.apply(unidecode)

    datos_inegi["municipio"] = datos_inegi.municipio.apply(str.lower)
    datos_inegi["municipio"] = datos_inegi.municipio.apply(unidecode)

    datos_inegi.loc[datos_inegi.estado == "mexico", "estado"] = "estado de mexico"

    return datos_inegi

In [11]:
datos_inegi_1 = pd.read_csv("./data/catalogo_inegi_clean.csv")
datos_inegi_1 = datos_inegi_1[["NOM_ENT", "NOM_MUN"]].drop_duplicates().rename(
    columns={"NOM_ENT": "estado", "NOM_MUN": "municipio"}
)

datos_inegi_1 = clean_datos_inegi(datos_inegi_1)
datos_inegi_1["exists_inegi_1"] = True

datos_inegi_1.shape

(2473, 3)

In [12]:

datos_inegi_2 = pd.read_csv("./data/catalogo_inegi_seccion_electoral.csv")
datos_inegi_2 = datos_inegi_2[["NOM_ENT", "NOM_MUN"]].drop_duplicates().rename(
    columns={"NOM_ENT": "estado", "NOM_MUN": "municipio"}
)

datos_inegi_2 = clean_datos_inegi(datos_inegi_2)

datos_inegi_2["exists_inegi_2"] = True

datos_inegi_2.shape

(2464, 3)

In [13]:
datos_inegi = datos_inegi_1.merge(
    datos_inegi_2,
    on=["municipio", "estado"],
    how="outer"
)
datos_inegi.shape

(2473, 4)

In [14]:
datos_inegi.isna().sum()

estado            0
municipio         0
exists_inegi_1    0
exists_inegi_2    9
dtype: int64

In [15]:
estados = datos_inegi.estado.unique().tolist()

# add extra estate
estados.append("distrito federal")

# first add the unique named municipios
municipios = datos_inegi.loc[~datos_inegi.municipio.isin(estados)].municipio.unique().tolist()

# add different spellings
municipios.append("praxedis g. de guerrero")
municipios.append("coyame de sotol")

# add municipios that have the same name as a state
municipios.extend(datos_inegi.loc[datos_inegi.municipio.isin(estados)].municipio.unique())

inegi_replacements = {
    "estate": {
        "names_list": estados,
        "base_regex": "[ ,](estados? de )?{name_value}[ ,.]",
        "replacement": " [ESTADO] "
    },
    
    "city": {
        "names_list": municipios,
        "base_regex": "[ ,](municipios? de )?{name_value}[ ,.]",
        "replacement": " [MUNICIPIO] "
    }
}

In [16]:
def explode_topics_data(publications_df: pd.DataFrame) -> pd.DataFrame:
    # explode each model topics into it's own row
    topics_df = publications_df.explode("topics_distribution").reset_index(
        drop = True
    )
    
    # normalize model's topic info into it's own column
    topics_df = topics_df.join(
        pd.json_normalize(topics_df.topics_distribution)
    ).drop(
        columns = ["topics_distribution"]
    )
    
    # explode each topic into it's own row
    topics_df = topics_df.explode("topics").reset_index(drop = True)
    
    # explode each topic's info into columns
    topics_df = topics_df.join(
        pd.json_normalize(topics_df.topics)
    ).drop(
        columns = ["topics", "model_id"]
    )
    
    # sort by topic's score
    topics_df = topics_df.sort_values(by="score", ascending=False).reset_index(drop=True)

    return topics_df

## Limpiar iniciativas

In [33]:
initiatives_cursor = conn.unique_publications.find({"type": "iniciativa"}, select_columns)
initiatives_df = pd.DataFrame(initiatives_cursor)
initiatives_df.shape

(5686, 3)

In [34]:
initiatives_df.head(2)

,_id,original_summary,topics_distribution
0,00024466b3c4102e89e26260242f86f8,Propone establecer que los servicios de salud ...,"[{'model_id': 'normalized_text_12', 'topics': ..."
1,00069181c1c5e63e833a2dc3d6798e9a,Propone establecer que para la obtención de la...,"[{'model_id': 'normalized_text_12', 'topics': ..."


In [28]:
initiatives_topics_df = explode_topics_data(initiatives_df)
initiatives_topics_df.shape

(25986, 4)

In [29]:
# keep representative topics
initiatives_topics_df = initiatives_topics_df.loc[initiatives_topics_df.score >= 0.5]
initiatives_topics_df.shape

(2497, 4)

### Exploración por topics

In [30]:
initiatives_topics_df.groupby("topic").agg({
    "_id": "count"
}).reset_index().sort_values(by="_id", ascending=False)

,topic,_id
0,0,663
4,4,446
8,8,351
10,10,259
6,6,229
1,1,204
7,7,136
3,3,108
5,5,69
11,11,14


In [31]:
topic = 2
num_documents = 20

filtered_df = initiatives_topics_df.loc[initiatives_topics_df.topic == topic]

num_row = 0
for _, row in filtered_df.iterrows(): 
    if num_row >= num_documents:
        break
    
    print(f"({row['_id']}) – {round(row['score'], 4)}")
    print("---"*5)
    print(row.original_summary)
    print("======="*10)
    
    num_row += 1

(253fea0c2c8f5eb5ec081d11e76354db) – 0.9765
---------------
Propone que, para los municipios de Coyame del Sotol, Ojinaga y Manuel Benavides, del Estado de Chihuahua, acorde a sus correlativos a su franja fronteriza, se aplico el meridiano 90 grados al oeste de Greenwich y para el Estado de Baja California, así como para los municipios de Janos, Ascensión, Juárez, Praxedis G. Guerrero y Guadalupe, del Estado de Chihuahua, acorde a sus correlativos a su franja fronteriza, se aplica el meridiano 105 grados al oeste de Greenwich.
(0e3953d9b0882adb031d026c9f7bf731) – 0.8472
---------------
Propone que la Ciudad de México sea una entidad federativa libre y soberana.
(c1e483c621cb470de2e3585f1c886752) – 0.6835
---------------
Propone establecer que la Zona Pacífico también comprenderá los municipios de Janos, Ascensión, Juárez, Praxedis G. de Guerrero y Guadalupe del Estado de Chihuahua, lo que implicará que su horario se determine conforme al paralelo 105 en lugar de 90 grados al oeste de G

### Replace common proper names

In [385]:
context_word = "soberanía"

matches = initiatives_df.loc[
    initiatives_df.original_summary.str.contains(context_word)
].original_summary.apply(get_context, context_word=context_word, before_word=True)

set().union(*matches)

{'Propone establecer una soberanía',
 'considerada por esta soberanía',
 'contribución a la soberanía',
 'división implica la soberanía',
 'expeler el amparo soberanía',
 'fortalecimiento de la soberanía',
 'la autonomía y soberanía',
 'la seguridad y soberanía'}

In [32]:
initiative_replacements = [
    {"replacement_regex": "(honorables? )?C[aá]maras? de Senador[ae]s( y Senador[ae]s)?", "replacement": "Camara_Senadores"},
    {"replacement_regex": "(Comisión Permanente del |Cámaras del |Honorable )?Congreso de la Unión", "replacement": "Congreso_Union"},
    {"replacement_regex": "Senado de la República", "replacement": "Senado_Republica"},
    {"replacement_regex": "Secretaría de", "replacement": ""},
     {"replacement_regex": "(esta|una) soberanía", "replacement": "[AUTORIDAD]"},
    {"replacement_regex": "(honorables? )?C[aá]maras? de Diputad[ao]s( y Diputad[ao]s)?", "replacement": "Camara_Diputados"},

]

In [35]:
initiatives_df["clean_summary"] = initiatives_df.original_summary
for replacement_data in initiative_replacements:
    initiatives_df["clean_summary"] = initiatives_df.clean_summary.apply(c.replace_string, **replacement_data)

In [36]:
text = initiatives_df.loc[
    initiatives_df.original_summary.str.contains("honorables Cámara de Senadores")
].clean_summary.iloc[0]
text

'Propone la inscripción de letras de oro en el Muro de Honor del Salón de sesiones de la Camara_Senadores la leyenda “A las sufragistas que nunca se rindieron ni abandonaron la lucha por la libertad”. En dicho reconocimiento se pretende visibilizará la lucha de las mujeres por la conquista de los derechos políticoelectorales.'

### Replace state + city names

In [37]:
clean_state_summaries = []
processing_errors = []

for i, original_text in tqdm(initiatives_df.clean_summary.items(), total=initiatives_df.shape[0]):
    try:
        clean_state_summaries.append(
            c.replace_places_by_flag(original_text, inegi_replacements=inegi_replacements)
        )
    except Exception as ex:
        # save error in list
        clean_state_summaries.append("[ERROR]")
        processing_errors.append((i, original_text, str(ex)))

# save summaries
initiatives_df["clean_state_summary"] = clean_state_summaries

100%|██████████| 5686/5686 [01:20<00:00, 70.98it/s] 


In [38]:
len(processing_errors)

0

In [41]:
# 103 w/original approach
# 8 w/new approach
replaced_strings = initiatives_df.loc[
    (initiatives_df.clean_state_summary.str.contains("\[MUNICIPIO\]"))
    | (initiatives_df.clean_state_summary.str.contains("\[ESTADO\]"))
]
replaced_strings.shape

(134, 5)

In [43]:
for _, row in replaced_strings.iterrows():
    print(row.original_summary)
    print("---")
    print(row.clean_state_summary)
    print("====="*10)

Propone que el Ministerio Público y los policías procederán de oficio o a petición de parte con el inicio de la indagatoria por los delitos en materia de trata de personas, inmediatamente después de que tengan conocimiento de éstos por cualquier medio formal o informal, bastando la noticia criminal. El Ministerio Público deberá contar con especialización en el tema de trata de personas y perspectiva de género. Las autoridades responsables de atender a las víctimas del delito en los ámbitos federal, las entidades federativas, los municipios y las demarcaciones territoriales de la Ciudad de México, n sus respectivos ámbitos de competencia, que deberán contar con especialización en el tema de trata de personas y perspectiva de género. Lo referente a la pena, propone imponer una pena de 25 a 45 años de prisión y de 2 mil 100 a 30 mil días multa, a quien, con la finalidad de explotar sexualmente a su pareja o descendientes de ésta, establezca alguna relación sentimental análoga bajo el enga

### Update data in mongo

In [44]:
updated_data = initiatives_df[["_id", "clean_summary", "clean_state_summary"]]
updated_data.shape

(5686, 3)

In [45]:
db.batch_update_publications(
    publications = updated_data.to_dict(orient="records"),
    table_name = "unique_publications", 
    conn = conn
)

## Limpiar proposiciones

* (ÚNICO/PRIMERO/.../TERCERO) El Senado de la República, ... exhorta/solicita,? respetuosamente,?
* Gobierno de .*
* (Palacio de )?Gobierno del Estado de .*
* [Ee]stado de .*
* plazo de .* días/meses
* se cumpla con la reforma a los artículos 76 y 78 de la Ley Federal del Trabajo
* [a-z])
* formula respetuoso exhorto

In [47]:
# 3 665
proposals_cursor = conn.unique_publications.find({"type": "proposicion"}, select_columns)
proposals_df = pd.DataFrame(proposals_cursor)
proposals_df.shape

(3665, 3)

### Explorar topics

In [52]:
proposals_topics_df = explode_topics_data(proposals_df)
proposals_topics_df.shape

(18387, 4)

In [56]:

# keep representative topics
proposals_topics_df = proposals_topics_df.loc[proposals_topics_df.score >= 0.3]
proposals_topics_df.shape

(3963, 4)

In [59]:
proposals_topics_df.groupby("topic").agg(
    {"_id": "count"}
).reset_index().sort_values(by="_id", ascending=False)

,topic,_id
7,7,1163
2,2,960
10,10,310
1,1,272
3,3,253
5,5,209
4,4,194
11,11,172
0,0,147
9,9,128


In [105]:
topic = 5
num_documents = 20

filtered_df = proposals_topics_df.loc[proposals_topics_df.topic == topic]

num_row = 0
for _, row in filtered_df.iterrows(): 
    if num_row >= num_documents:
        break
    
    print(f"({row['_id']}) – {round(row['score'], 4)}")
    print("---"*5)
    print(row.original_summary)
    print("======="*10)
    
    num_row += 1

(f468d908ce850363ee1e5c1e0adc5d73) – 0.9684
---------------
ÚNICO. La Comisión Permanente del Honorable Congreso de la Unión exhorta respetuosamente a la Secretaría de Hacienda y Crédito Público a que rinda a esta Soberanía, un informe pormenorizado, en un plazo de quince días hábiles, donde explique la situación que guarda el ejercicio del gasto público respecto del Presupuesto de Egresos de la Federación para 2019 e informe sobre la situación económica nacional.
(ac3eb2f4b2e503d1aba51bee08513bb1) – 0.9647
---------------
ÚNICO.- La Comisión Permanente del H. Congreso de la Unión exhorta respetuosamente a la Secretaría de Relaciones Exteriores a que envíe un informe a esta Soberanía, relativo a la postura de la dependencia y de la representación diplomática respecto de las presiones y expresiones emitidas por el gobernador del Estado de Texas, Estados Unidos de América, que puedan constituir violaciones a la soberanía nacional de nuestro país.
(3e66261e55dd9d7c527d0d8bf45172e0) – 0.95

### Quitar palabras repetidas

In [129]:
context_word = "informe"
matches = proposals_df.loc[
    proposals_df.original_summary.str.contains(context_word)
].original_summary.apply(
    c.get_word_context, 
    context_word=context_word,
    before_words=0,
    after_words=1
)

set().union(*matches)

{' informe ',
 ' informe a ',
 ' informe acerca ',
 ' informe actualizado ',
 ' informe aeta ',
 ' informe al ',
 ' informe ante ',
 ' informe cerca ',
 ' informe como ',
 ' informe con ',
 ' informe considerará ',
 ' informe contenga ',
 ' informe cual ',
 ' informe cuantas ',
 ' informe cuál ',
 ' informe cuáles ',
 ' informe cuántas ',
 ' informe cómo ',
 ' informe de ',
 ' informe deberá ',
 ' informe del ',
 ' informe desglosado ',
 ' informe destallado ',
 ' informe detalladamente ',
 ' informe detallado ',
 ' informe detallado, ',
 ' informe donde ',
 ' informe el ',
 ' informe en ',
 ' informe especial ',
 ' informe esta ',
 ' informe exhaustivo ',
 ' informe final ',
 ' informe final, ',
 ' informe financiero ',
 ' informe formal ',
 ' informe integral ',
 ' informe la ',
 ' informe las ',
 ' informe los ',
 ' informe o ',
 ' informe oficial ',
 ' informe particularmente ',
 ' informe pertinente ',
 ' informe policial ',
 ' informe por ',
 ' informe pormenorizadamente ',
 ' in

In [123]:
propositions_replacements = [
    {"replacement_regex": "(honorables? )?C[aá]maras? de Senador[ae]s( y Senador[ae]s)?", "replacement": "Camara_Senadores"},
    {"replacement_regex": "(Comisión Permanente del |Cámaras del )?(H\. |Honorable )?Congreso de la Unión", "replacement": "Congreso_Union"},
    {"replacement_regex": "Senado de la Rep[uú]blica", "replacement": "Senado_Republica"},
    {"replacement_regex": "Secretarías? de", "replacement": ""},
    {"replacement_regex": "([Ee]xhortar?|EXHORTA|exhorto|[Ss]olicit[oa]) [Rr]espetuosamente", "replacement": ""},
    {"replacement_regex": "(esta|una) ([Hh]onorable |H\. )?[sS]oberanía", "replacement": "[AUTORIDAD]"},
    {"replacement_regex": "(honorables? )?C[aá]maras? de Diputad[ao]s( y Diputad[ao]s)?", "replacement": "Camara_Diputados"},
    {"replacement_regex": "Diario Oficial de la Federación", "replacement": "DoF"},
    {"replacement_regex": "([ÚU]NICO|PRIMERO|SEGUNDO|TERCERO|CUARTO|QUINTO)\.", "replacement": ""},
    {"replacement_regex": "plazo (no mayor |que no exceda )?(de|a) .*? días hábiles", "replacement": "[PLAZO_ENTREGA]"},
    {"replacement_regex": "(t[eé]rmino|periodo) de .*? días hábiles", "replacement": "[PLAZO_ENTREGA]"},
]

In [124]:
proposals_df["clean_summary"] = proposals_df.original_summary
for replacement_data in tqdm(propositions_replacements, total=len(propositions_replacements)):
    proposals_df["clean_summary"] = proposals_df.clean_summary.apply(c.replace_string, **replacement_data)

100%|██████████| 11/11 [00:00<00:00, 11.35it/s]


In [125]:
replaced_rows = proposals_df.loc[
    proposals_df.original_summary.str.contains("días hábiles")
]

In [126]:
num_row = 1
replaced_rows.iloc[num_row].original_summary

'PRIMERO. La Comisión Permanente del H. Congreso de la Unión exhorta respetuosamente a la Comisión Nacional de Pesca y Acuacultura a que, en un término de cinco días hábiles después de publicarse este exhorto, presente ante esta soberanía un informe destallado sobre los apoyos otorgados a los pescadores ribereños del Estado de Campeche para: a) mejorar físicamente sus embarcaciones; b) promover el cambio a motores marinos modernos; e) proveerlos de artes de pesca amigables con el medio ambiente; d) conocer sobre los apoyos que les han sido entregados en tecnologías de información y comunicación, y e) saber sobre las acciones realizadas para eficientar los espacios de las embarcaciones.  SEGUNDO. La Comisión Permanente del H. Congreso de la Unión exhorta respetuosamente a la Comisión Nacional de Pesca y Acuacultura a que, en un término de cinco días hábiles después de publicarse este exhorto, presente ante esta soberanía las disposiciones para garantizar que los apoyos a los pescadores 

In [127]:
replaced_rows.iloc[num_row].clean_summary

' La Congreso_Union  a la Comisión Nacional de Pesca y Acuacultura a que, en un [PLAZO_ENTREGA] después de publicarse este exhorto, presente ante [AUTORIDAD] un informe destallado sobre los apoyos otorgados a los pescadores ribereños del Estado de Campeche para: a) mejorar físicamente sus embarcaciones; b) promover el cambio a motores marinos modernos; e) proveerlos de artes de pesca amigables con el medio ambiente; d) conocer sobre los apoyos que les han sido entregados en tecnologías de información y comunicación, y e) saber sobre las acciones realizadas para eficientar los espacios de las embarcaciones.   La Congreso_Union  a la Comisión Nacional de Pesca y Acuacultura a que, en un [PLAZO_ENTREGA] después de publicarse este exhorto, presente ante [AUTORIDAD] las disposiciones para garantizar que los apoyos a los pescadores de Campeche, consistentes en: a) adquisición de embarcaciones y equipos de conservación; b) la adquisición de equipos e infraestructura para la producción más lim

### Clean states data

In [130]:
clean_state_summaries = []
processing_errors = []

for i, original_text in tqdm(proposals_df.clean_summary.items(), total=proposals_df.shape[0]):
    try:
        clean_state_summaries.append(
            c.replace_places_by_flag(original_text, inegi_replacements=inegi_replacements)
        )
    except Exception as ex:
        # save error in list
        clean_state_summaries.append("[ERROR]")
        processing_errors.append((i, original_text, str(ex)))

# save summaries
proposals_df["clean_state_summary"] = clean_state_summaries

100%|██████████| 3665/3665 [07:46<00:00,  7.86it/s]


In [131]:
len(processing_errors)


0

In [134]:
replaced_strings = proposals_df.loc[
    (proposals_df.clean_state_summary.str.contains("\[MUNICIPIO\]"))
    | (proposals_df.clean_state_summary.str.contains("\[ESTADO\]"))
]
replaced_strings.shape

(1297, 5)

In [135]:
for _, row in replaced_strings.iterrows():
    print(row.original_summary)
    print("---")
    print(row.clean_state_summary)
    print("====="*10)

PRIMERO.- La Comisión Permanente del H. Congreso de la Unión exhorta, respetuosamente, al Instituto Nacional de Migración, a la Guardia Nacional, a los gobiernos de los estados de Chihuahua, Coahuila, Nuevo León y Tamaulipas, así como a los gobiernos de los municipios de las entidades colindantes con el Río Bravo para que en coordinación detecten en donde se encuentran las boyas colocadas por el gobierno texano y coloquen señalizaciones que alerten a los migrantes de la existencia de las boyas con púas, así como que realicen una campaña masiva de difusión de los riesgos en las zonas en donde el muro flotante ha sido construido, para salvaguardar su vida e integridad física.  SEGUNDO.- La Comisión Permanente del H. Congreso de la Unión rechaza y condena enérgicamente la colocación de boyas revestidas de púas en el Río Bravo por parte del Gobierno del Estado de Texas, en los Estados Unidos de América, al representar una violación grave a los derechos humanos de las personas migrantes, po

### Update mongo

In [136]:
updated_data = proposals_df[["_id", "clean_summary", "clean_state_summary"]]
updated_data.shape

(3665, 3)

In [137]:
db.batch_update_publications(
    publications = updated_data.to_dict(orient="records"),
    table_name = "unique_publications", 
    conn = conn
)